# Analysis of Salary Packaging for Electric Vehicles Australia

Let's look at a salary packaging quote for a Tesla Model 3 and find out how much of the tax benefit the middle man is able to cream off because of the Government's dummer-than-a-bag-of-hammers policy design.

In [3]:
#r "nuget:FinancialSharp"

Installed Packages FinancialSharp, 1.0.1

In [4]:
let CAR_ONLY_PAYMENT_PRE_TAX_MONTHLY = 1107.13 // From SGfleet quote
let TESLA_MODEL3_COST = 63900.00 // From Tesla website
let LEASE_TERM_MONTHS = 60 // From SGfleet quote
let CRIPPLING_BALLOON_PAYMENT = 19344.16 // From SGfleet quote and Simpsons: https://youtu.be/su_3x4jsv6w


Let's find out the effective rate of interest the lessor is getting on this finance.  The lessor has to pay the Tesla cost up front, then gets lease payments each month plus balloon payment.  The balloon payment is added to last lease payment.
For an IRR calc, it doesn't matter whether payments are pre/post tax or incl GST - it just has to be consistent.  So we use pre-tax, incl GST.
In reality the lessor probably gets the Tesla at a slight fleet discount, not reflected here.

In [5]:
let allCashflowsForFinancier = [-TESLA_MODEL3_COST] @ ([1..LEASE_TERM_MONTHS-1] |> List.map (fun _ -> CAR_ONLY_PAYMENT_PRE_TAX_MONTHLY))  @ [CAR_ONLY_PAYMENT_PRE_TAX_MONTHLY + CRIPPLING_BALLOON_PAYMENT] 

let cashFlowsArray = allCashflowsForFinancier |> Seq.ofList |> Seq.map (fun i -> double i)
let irrMonthly= FinancialSharp.Financial.Irr(cashFlowsArray) 
let irrAnnual = irrMonthly |> Option.map (fun i -> (1.0 + i)**12.0 - 1.0) 
irrAnnual



Value
0.10449672114077369


Nice return for lessor!

Let's work out what the discounted cost would be if you just bought the damn Tesla with cash borrowed at a mortgage rate, like from an offset account.
We use that as the discount rate.
We assume you make the same payments as you would have under the lease agreement.
We calculate the outstanding balance at the end of the term of the lease if you had made the same monthly payments as in the lease.  This is the same as the residual in the lease would be if it offered the same rate of finance that you could get via your offset account.

In [6]:
let ALTERNATIVE_FINANCE_RATE = 0.0482 // eg, residentially secured offset account rate, annual interest rate, not 'comparison rate'
let ANNUAL_TAX_BENEFIT = 6951.00 // from sgFleet quote

let rec calcEndingBalance(balance: float, periodNumber: int, maxPeriods: int, repayment: float, interestRate: float) =
    
    let calcPeriodEndBalance(startingBalance: float, repayment: float) = (startingBalance * (1.0 + interestRate)) - repayment // interest accrues on balance at beginning of period, repayment is at end
        
    match periodNumber >= maxPeriods with
    | true -> balance
    | false ->
        let currentBalance = calcPeriodEndBalance(balance, repayment)
        calcEndingBalance(currentBalance, periodNumber + 1, maxPeriods, repayment, interestRate) 

let monthlyRateOfAltnativeFinance = ALTERNATIVE_FINANCE_RATE / 12.0;

let endingBalanceIfPrivatelyFinanced = calcEndingBalance(TESLA_MODEL3_COST, 0, LEASE_TERM_MONTHS, CAR_ONLY_PAYMENT_PRE_TAX_MONTHLY, (monthlyRateOfAltnativeFinance)) 
endingBalanceIfPrivatelyFinanced


6328.054050190987

So what is the discounted cost of the additional finance and other fees in the lease agreement?  The lease agreement also has some inflated fees for insurance and rego.  There is also a management fee and a contingency fee and some other fees.  We can model these as a stream of additional payments.  We should use the after tax values because under the lease agreement they would be FBT exempt.  

In [7]:
let additonalFeesAnnualEstimateAfterTax =  1000.0 // sgfleet charges management fee, overpriced insurance, padded rego cost, a few other odds and ends

let diffBetweenPrivateFinaceResidualAndSgFleetResidual = CRIPPLING_BALLOON_PAYMENT - endingBalanceIfPrivatelyFinanced 
let npvOfAdditionalFinanceCost = diffBetweenPrivateFinaceResidualAndSgFleetResidual / (1.0 + monthlyRateOfAltnativeFinance)**(float LEASE_TERM_MONTHS) 
let npvOfAdditionalFees = FinancialSharp.Financial.Npv(ALTERNATIVE_FINANCE_RATE, seq {for _ in 1 ..5 -> additonalFeesAnnualEstimateAfterTax}) 
let totalAdditionalNpvOfSgFleet = npvOfAdditionalFinanceCost + npvOfAdditionalFees 


The discounted cost of the additional finance and other fees in the lease agreement:

In [8]:
totalAdditionalNpvOfSgFleet

14794.363879078797

What is the discounted value of the tax benefit?  We just discount that annually because you only get it anually.

In [12]:
let npvOfTaxBenefit = FinancialSharp.Financial.Npv(ALTERNATIVE_FINANCE_RATE, seq {for _ in 1 .. 5 -> ANNUAL_TAX_BENEFIT}) 
npvOfTaxBenefit

31702.31062394445

The proportion of the tax benefit that goes to the middle men:

In [11]:
let middleMenCut =  totalAdditionalNpvOfSgFleet /  npvOfTaxBenefit
middleMenCut

0.4666651606115031